In [1]:
import pandas as pd
from openai import OpenAI
from datetime import datetime
import json

In [ ]:
# Initialize OpenAI client with your API key
import openai
openai.api_key = "key_here"  # Replace with your actual API key
client = openai.OpenAI(api_key="key_here")


Cookie Sheet processing

In [ ]:
# Load template and input Excel files
#template = pd.ExcelFile('template.xlsx')
input_file_name = "APM0006263 - CFFS-Manual-Cookie-Scan-Report.xlsx"
output_file_name = "Done-APM0006263 - CFFS-Manual-Cookie-Scan-Report.xlsx"
input_data = pd.read_excel(input_file_name, sheet_name=None)  # read all sheets into a dict of DataFrames
print("File extraction done")

# Process Cookies sheet
try:
    cookies_df = input_data.get('Cookies')
    #Ensure required columns exist
    for col in ['Category', 'Description']:
          if col not in cookies_df.columns:
                 cookies_df[col] = '' #Add new column if not exist
    print(cookies_df.head())
except:
    print("The document does not have cookie sheet")

import time
#If needed include the key here
openai.api_key = "key_here"  # Replace with your actual API key
client = openai.OpenAI(api_key="key_here")

cookies_df['Description'] = cookies_df['Description'].astype(str)
cookies_df['Category'] = cookies_df['Category'].astype(str)

# Iterate through each row in Cookies sheet
for i, row in cookies_df.iterrows():
    name = row['Name']  # get the cookie name
    expiration = row['Expiration']  # get expiration value
    lifespan = row['Lifespan']  # get lifespan value
    # Check and format expiration date or lifespan
    if expiration == 'Infinity':
            # If expiration is Infinity, set lifespan as Session
            cookies_df.at[i, 'Lifespan'] = 'Session'    
    elif lifespan == 'Session':
        # If lifespan is Session, set the expiration as Infinity
        cookies_df.at[i, 'Expiration'] = 'Infinity'
    elif pd.notnull(expiration) and expiration != 'Infinity': 
        # Format valid date to required ISO format
        expiration = str(expiration).strip()  # remove spaces
        expiration = expiration.replace('/', '-')  # replace slashes with dashes if needed
        expiration = pd.to_datetime(expiration,errors='coerce').strftime('%Y-%m-%dT%H:%M:%S.000Z')    
        print("log:",expiration)
        cookies_df.at[i, 'Expiration'] = expiration
        cookies_df.at[i, 'Lifespan'] = 'Permanent'
# Call ChatGPT to categorize cookie and generate description
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a cookie categorizer."},
            {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for cookie name {name} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {name}"}],
        temperature=0 )
    text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
    print("GPT:",text)
    result = json.loads(text)  # parse JSON
    cookies_df.at[i, 'Category'] = result.get('Category', '').strip()
    cookies_df.at[i, 'Description'] = result.get('Description', '').strip()
    sucess = True #done!
    print(f"Got the category done!")

import openpyxl
# Find lowest expiration timestamp

# Extract the expiration dates after 2000 year
cookies_df['Expiration'] = pd.to_datetime(cookies_df['Expiration'],errors = 'coerce')
valid_dates = cookies_df[cookies_df['Expiration'].dt.year>=2000]['Expiration']
print(f"all dates:{valid_dates}")

# Find the lowest year to find the cookie scan date
min_date = valid_dates.min()
print(f"Minimum date is:{min_date}")

# convert the lowest date into iSO standards
if pd.notna(min_date):
     manual_scan_date = min_date.strftime('%Y-%m-%dT%H:%M:%S.000Z')
     print(f"manual_scan_date is:{manual_scan_date}")
else:
     manual_scan_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S.000Z')
     print(f"min_date is NaT. Cannot format, so using current date {manual_scan_date}")
    
# Format the expiration date format, only for the dates.
cookies_df['Expiration'] = cookies_df['Expiration'].where(
    cookies_df['Expiration'].apply(lambda x: isinstance(x,pd.Timestamp)), pd.NaT
).apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S.000Z") if pd.notna(x) else 'Infinity')

# print("DataFrame shape:",cookies_df.shape)
# print("Column names:",cookies_df.columns.tolist())
# print("Last 5 rows:\n",cookies_df.head())

#Avoid duplicating footer rows and Append Timestamp and Website info
if not cookies_df.iloc[-3:, 0].str.contains("Time stamp of Manual Scan run").any():     
    cookies_df.loc[len(cookies_df)] = [''] * len(cookies_df.columns)     
    cookies_df.loc[len(cookies_df)] = ['Time stamp of Manual Scan run', manual_scan_date] + [''] * (len(cookies_df.columns) - 2)     
    domain_value = cookies_df['Domain'].iloc[0] if 'Domain' in cookies_df.columns else ''     
    cookies_df.loc[len(cookies_df)] = ['Website', domain_value] + [''] * (len(cookies_df.columns) - 2) 

# Convert all to string before writing to Excel 
cookies_df = cookies_df.astype(str)

# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl') as writer:
    cookies_df.to_excel(writer, sheet_name='Cookies', index=False)

print(f"Done,{output_file_name}")

File extraction done
                 Domain               Name                     Value  \
0  cffs.edsced-sltc.com  ASP.NET_SessionId  y1xq3ccirdolwquu2xmqnb3z   

  Expiration Lifespan  ThirdParty  Path Secure  HTTP Only  Source  \
0    Session  Session       False   NaN    Yes      False  script   

                         Set By                          Page Category  \
0  https://cffs.edsced-sltc.com  https://cffs.edsced-sltc.com            

  Description  
0              
GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Used to maintain an anonymized user session by the server."
}
Got the category done!
all dates:Series([], Name: Expiration, dtype: datetime64[ns])
Minimum date is:NaT
min_date is NaT. Cannot format, so using current date 2025-07-17T12:04:43.000Z
Done,Done-APM0006263 - CFFS-Manual-Cookie-Scan-Report.xlsx


Local Sheet processing

In [ ]:
# Process Local sheet
try:
    local_df = input_data.get('Local Storage')
    
    for col in ['Category', 'Description']:
        if col not in local_df.columns:
            local_df[col] = '' #Add new column if not exist
    
    print(local_df.head())
except:
    print("There is no Local Storage sheet exist")
#Ensure required columns exist


                            Domain  \
0  dcffinracompliance.deloitte.com   
1  dcffinracompliance.deloitte.com   
2  dcffinracompliance.deloitte.com   
3  dcffinracompliance.deloitte.com   
4  dcffinracompliance.deloitte.com   

                                                 Key  \
0  msal.556f8b97-2187-4ed5-beda-6f24aa153330.acti...   
1  fc0dd5e1-b923-4f25-b4e2-a1ed761c7e07.36da45f1-...   
2  fc0dd5e1-b923-4f25-b4e2-a1ed761c7e07.36da45f1-...   
3                                        UserEmailID   
4                                       isAuthorized   

                     Category  Description  
0  Strictly necessary cookies          NaN  
1  Strictly necessary cookies          NaN  
2  Strictly necessary cookies          NaN  
3  Strictly necessary cookies          NaN  
4  Strictly necessary cookies          NaN  


In [10]:
# Process Local Storage sheet
local_df = input_data.get('Local Storage')
if local_df is not None:
    for i, row in local_df.iterrows():
        key = row['Key']  # key name
 
        # Call ChatGPT to get category and description
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cookie categorizer."},
                {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for local storage cookie name {key} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {key}"}],
            temperature=0 )
        text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
        print("GPT:",text)
        result = json.loads(text)  # parse JSON
        local_df.at[i, 'Category'] = result.get('Category', '').strip()
        local_df.at[i, 'Description'] = result.get('Description', '').strip()
        sucess = True #done!
        print(f"Got the category done for local storage")

GPT: {
  "Category": "Strictly Necessary Cookies",
  "Description": "Essential for user authentication and account management."
}
Got the category done for local storage


D:\Users\skotteeswaran\AppData\Local\Temp\ipykernel_5160\1917192650.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Essential for user authentication and account management.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  local_df.at[i, 'Description'] = result.get('Description', '').strip()


GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Essential for user authentication and access to user profile and directory information."
}
Got the category done for local storage
GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Essential for user authentication and security purposes."
}
Got the category done for local storage
GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Stores the user's email ID for essential website functionality."
}
Got the category done for local storage
GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Essential for user authentication and security purposes."
}
Got the category done for local storage
GPT: {
  "Category": "Functional Cookies",
  "Description": "Stores active account filters for user convenience."
}
Got the category done for local storage
GPT: {
  "Category": "Strictly Necessary Cookies",
  "Description": "Essential for user authentication and securit

In [11]:
# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl', mode='a', if_sheet_exists ='replace') as writer:
    local_df.to_excel(writer, sheet_name='Local Storage', index=False)

print("Done!")

Done!


In [ ]:
# Process Cookies sheet
try:
    session_df = input_data.get('Session Storage')
    
    for col in ['Category', 'Description']:
        if col not in session_df.columns:
            session_df[col] = '' #Add new column if not exist
    
    print(session_df.head())
except:
    print("There is no Session Storage sheet exist")
#Ensure required columns exist


In [ ]:
# Process Session Storage sheet
session_df = input_data.get('Session Storage')
if session_df is not None:
    for i, row in session_df.iterrows():
        key = row['Key']  # key name
 
        # Call ChatGPT to get category and description
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cookie categorizer."},
                {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for session storage cookie name {key} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {key}"}],
            temperature=0 )
        text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
        print("GPT:",text)
        result = json.loads(text)  # parse JSON
        session_df.at[i, 'Category'] = result.get('Category', '').strip()
        session_df.at[i, 'Description'] = result.get('Description', '').strip()
        sucess = True #done!
        print(f"Got the category done for Session storage")

In [ ]:
# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl', mode='a', if_sheet_exists ='replace') as writer:
    session_df.to_excel(writer, sheet_name='Session Storage', index=False)

print("Done!")